<a href="https://colab.research.google.com/github/Akende1/code-unza25-csc4792-project_team_27-repository/blob/main/code_unza25_csc4792_project_team_27_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Business Understanding

## Problem Statement

Wikipedia serves as a widely accessible knowledge repository, however the quality and completeness of articles vary. For Zambian topics, many articles remain underdeveloped, lacking depth, proper structure, sufficient references, and multimedia elements. This limits the availability of reliable, comprehensive information about Zambia for both local and global audiences.

Currently the process of identifying incomplete Zambian articles is largely manual, relying on editor intuition, quality assessments, and community discussions. This approach is time-consuming, inconsistent, and insufficient for  addressing the most critical content gaps. Without a scalable method to assess and prioritize articles, valuable editing efforts may be misdirected, leaving important topics underrepresented.

The core problem is the absence of an automated, consistent, and accurate system to determine and classify the completeness of Zambian Wikipedia pages. By analyzing text structure, metadata, and content coverage, such a system could classify articles according to Wikipedia’s quality scale and highlight key areas for improvement. This would enable editors to focus on the most impactful updates, improve the overall quality of Zambian content, and ensure that readers have access to well-developed, trustworthy information.

Solving this problem benefits multiple stakeholders: Wikipedia editors seeking guidance on where to contribute, WikiProject Zambia aiming to raise overall article standards, researchers and educators relying on accurate information, and the general public seeking a richer understanding of Zambia.